Importación de librerías

In [22]:
import pandas as pd
from Dataframes import contratos_df, pagos_df
import os

Carga de datos

In [23]:
data_dir = './data/'

proyectos_df = pd.read_csv(os.path.join(data_dir, 'proyectos.csv'))
# contratos_df = pd.read_csv(os.path.join(data_dir, 'contratos.csv'))
# pagos_df = pd.read_csv(os.path.join(data_dir, 'pagos.csv'))
ofertas_df = pd.read_csv(os.path.join(data_dir, 'ofertas.csv'))

Porcentaje de atraso de las obras públicas

In [24]:
contratos_df['ATRASO'] = (
    pd.to_datetime(contratos_df['FECHATERMINO']) - 
    pd.to_datetime(contratos_df['FECHATERMINOORIGINAL'])
).dt.days

obras_con_atraso = contratos_df[contratos_df['ATRASO'] > 0].shape[0]
total_obras = contratos_df.shape[0]
porcentaje_atraso = (obras_con_atraso / total_obras) * 100

print(f"Porcentaje de obras con atraso: {porcentaje_atraso:.2f}%")

Porcentaje de obras con atraso: 46.34%


Tipo de obra que tarda más en realizarse

In [25]:
contratos_df['DURACION'] = (
    pd.to_datetime(contratos_df['FECHATERMINO']) - 
    pd.to_datetime(contratos_df['FECHAINICIO'])
).dt.days

tipo_obra_mas_lenta = contratos_df.groupby('SERVICIO', observed=False)['DURACION'].mean().idxmax()

print(f"Tipo de obra que tarda más en realizarse: {tipo_obra_mas_lenta}")

Tipo de obra que tarda más en realizarse: VIALIDAD


Tipo de obra más cara

In [29]:
tipo_obra_mas_cara = contratos_df.groupby('SERVICIO', observed=False)['MONTOVIGENTE'].mean().idxmax()

print(f"Tipo de obra más cara: {tipo_obra_mas_cara}")

Tipo de obra más cara: VIALIDAD


Variación del presupuesto por año

In [27]:
pagos_df['FECHAPAGO'] = pd.to_datetime(pagos_df['FECHAPAGO'])
pagos_df['ANIO'] = pagos_df['FECHAPAGO'].dt.year

presupuesto_anual = pagos_df.groupby('ANIO')['TOTALPAGADO'].sum()

#Convertimos esto a un dataframe

presupuesto = {}

presupuesto['Año'] = presupuesto_anual.index
presupuesto['Presupuesto'] = presupuesto_anual.values

presupuesto = pd.DataFrame(presupuesto)

#Datos de la inflación son sacados de https://www.datosmundial.com/america/chile/inflacion.php

inflacion_acumulada = {
    2023: 1.00,  # El año base
    2022: 1.0758,  # 1 + 7.58% de inflación
    2021: 1.1252,  # 1 + 12.52% acumulado hasta 2021
    2020: 1.1552,  # 1 + 15.52% acumulado hasta 2020
    2019: 1.1810,  # 1 + 18.10% acumulado hasta 2019
    2018: 1.1940,  # 1 + 19.40% acumulado hasta 2018
    2017: 1.2196,  # 1 + 21.96% acumulado hasta 2017
    2016: 1.1844,  # 1 + 18.44% acumulado hasta 2016
    2015: 1.1456,  # 1 + 14.56% acumulado hasta 2015
    2014: 1.1148,  # 1 + 11.48% acumulado hasta 2014
    2013: 1.1929,  # 1 + 19.29% acumulado hasta 2013
    2012: 1.1636,  # 1 + 16.36% acumulado hasta 2012
    2011: 1.1482,  # 1 + 14.82% acumulado hasta 2011
    2010: 1.0706   # 1 + 7.06% acumulado hasta 2010
}

#aquí básicamente convierto todos los presupuestos de años anteriores a lo que valdría ese presupuesto hoy 

presupuesto= presupuesto.drop(14)

def poner_puntos(numero): # le doy los puntitos bonitos
    return "{:,}".format(numero).replace(",", ".")

for x in range(len(presupuesto)):
    presupuesto.loc[x, 'Presupuesto'] = poner_puntos(round(presupuesto.loc[x, 'Presupuesto'] * inflacion_acumulada[int(presupuesto.loc[x, 'Año'])]))

presupuesto

C:\Users\IMC\AppData\Local\Temp\ipykernel_6124\2816817020.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '266.086.711.295' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  presupuesto.loc[x, 'Presupuesto'] = poner_puntos(round(presupuesto.loc[x, 'Presupuesto'] * inflacion_acumulada[int(presupuesto.loc[x, 'Año'])]))


,Año,Presupuesto
0,2010,266.086.711.295
1,2011,666.881.355.330
2,2012,926.679.538.895
3,2013,1.119.624.563.279
4,2014,1.094.851.143.784
5,2015,1.274.747.523.418
6,2016,1.240.929.239.332
7,2017,1.300.141.038.243
8,2018,1.221.112.915.340
9,2019,1.433.943.441.655


Empresa más contratada

In [28]:
empresa_mas_contratada = pagos_df['NOMBREADJUDICADO'].value_counts().idxmax()

print(f"Empresa más contratada: {empresa_mas_contratada}")

Empresa más contratada: CONST. DE PAVIMENTOS ASFALTICOS BITUMIX S.A.


Promedio de atraso de las obras publicas

In [41]:
promedio_atraso = contratos_df[contratos_df.ATRASO > 0]['ATRASO'].mean()
print(f'El promedio de atraso de las obras publicas es de {round(promedio_atraso, 2)} diasa')

El promedio de atraso de las obras publicas es de 102.77 diasa
